In [1]:
import os
import numpy as np
import netCDF4
import datetime
import logging
import cmocean
import calendar
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [2]:
logger = logging.getLogger("ClimComparison")
logger.setLevel(logging.INFO)
logging.info("Sarting")

## Files and directories

In [3]:
datadir1 = "/home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/output/"
datadir2 = "/data/SeaDataCloud/NorthSea/Climato/"
figdir = "/home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/"
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [52]:
# Path to the products to be compared
# Old products
salinity_month_sdn = os.path.join(datadir2, "Salinity.19002015.4Danl.nc")
temperature_month_sdn = os.path.join(datadir2, "Temperature.19002015.4Danl.nc")

# Annual products
# New products, only SDC data
temperature_year_sdc0 = os.path.join(datadir1, "SDN/Water_body_Temperature_NorthSea.4Danl_annual.nc")
salinity_year_sdc0 = os.path.join(datadir1, "SDN/Water_body_Salinity_NorthSea.4Danl_annual.nc")
logger.info(os.path.exists(temperature_year_sdc1) & os.path.exists(salinity_year_sdc1))

# New products, SDc + WOD data, without coefficients derivative
temperature_year_sdc1 = os.path.join(datadir1, "Water_body_Temperature_NorthSea.4Danl_annual_merged.nc")
salinity_year_sdc1 = os.path.join(datadir1, "Water_body_Salinity_NorthSea.4Danl_annual_merged.nc")
logger.info(os.path.exists(temperature_year_sdc1) & os.path.exists(salinity_year_sdc1))

# New products with coefficient
temperature_year_sdc2 = os.path.join(datadir1, "Water_body_Temperature_NorthSea.4Danl_annual_merged_coeffderiv.nc")
salinity_year_sdc2 = os.path.join(datadir1, "Water_body_Salinity_NorthSea.4Danl_annual_merged_coeffderiv.nc")

INFO:ClimComparison:True
INFO:ClimComparison:True


## Reading & plotting functions

In [24]:
def read_climato(filename, varname):
    """
    Read the climatology from the netCDF file `filename`.
    """
    if varname.lower() == "temperature":
        std_name = "sea_water_temperature"
    elif varname.lower() == "salinity":
        std_name = "sea_water_salinity"
    
    with netCDF4.Dataset(filename, "r") as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        depth = nc.variables["depth"][:]
        ttime = nc.variables["time"][:]
        timeunits = nc.variables["time"].units
        dates = netCDF4.num2date(ttime, timeunits)
        try:
            field = nc.get_variables_by_attributes(standard_name=std_name)[0][:]
        except :
            field = nc.get_variables_by_attributes(long_name=varname.capitalize())[0][:]
        
    return lon, lat, depth, dates, field

In [50]:
def make_plot_2Dfield(lon1, lat1, lon2, lat2, field1, field2, figtitle, 
                      figname, subtitle1="",subtitle2="", **kwargs):
    
    if varname.lower() == "temperature":
        cb_label = "T ($^{\circ}$C)"
        vmin = 7.5
        vmax = 17.5
        cmap = cmocean.cm.thermal
    elif varname.lower() == "salinity":
        cb_label = "S"
        vmin = 30.
        vmax = 40.
        cmap = cmocean.cm.haline
    else:
        logger.error("Unknown variable")
    
    fig = plt.figure(figsize=(12, 6))
    ax1 = plt.subplot(121)
    m.pcolormesh(lon1, lat1, field1, latlon=True, 
                 cmap=cmap, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-5., lon1.max(), 3.), labels=[0,1,0,1], 
                    linewidth=0.5, zorder=2, fontsize=14)
    m.drawparallels(np.arange(48., lat1.max(), 2.), labels=[1,0,1,0], 
                    linewidth=0.5, zorder=2, fontsize=14)
    m.drawcoastlines(linewidth=0.2, zorder=4)
    m.fillcontinents(color=".75", zorder=3)
    cb = plt.colorbar(extend="both", shrink=.9)
    cb.set_label(cb_label, rotation=0, ha="left", fontsize=14)
    plt.title(subtitle1)

    plt.subplot(122)
    m.pcolormesh(lon2, lat2, field2, latlon=True,
                cmap=cmap, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-5., lon1.max(), 3.), labels=[0,1,0,1], 
                    linewidth=0.5, zorder=2, fontsize=14)
    m.drawparallels(np.arange(48., lat1.max(), 2.), labels=[1,0,1,0], 
                    linewidth=0.5, zorder=2, fontsize=14)
    m.drawcoastlines(linewidth=0.25, zorder=5)
    m.fillcontinents(color=".75", zorder=3)
    cb = plt.colorbar(extend="both", shrink=.9)
    cb.set_label(cb_label, rotation=0, ha="left", fontsize=14)
    plt.title(subtitle2)
    fig.suptitle(figtitle, fontsize=14)
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    #plt.show()
    plt.close()

## Comparisons
### 1. Field with and without coefficients

In [42]:
lon1, lat1, depth1, dates1, T1 = read_climato(temperature_year_sdc1, "temperature")
lon2, lat2, depth2, dates2, T2 = read_climato(temperature_year_sdc2, "temperature")
lon1, lat1, depth1, dates1, S1 = read_climato(salinity_year_sdc1, "salinity")
lon2, lat2, depth2, dates2, S2 = read_climato(salinity_year_sdc2, "salinity")

In [27]:
m = Basemap(projection='merc',llcrnrlat=lat1.min(), urcrnrlat=lat1.max(),
            llcrnrlon=-5.5, urcrnrlon=lon1.max(),lat_ts=lat1.mean(), resolution='i')

In [45]:
llon1, llat1 = np.meshgrid(lon1, lat1)
llon2, llat2 = np.meshgrid(lon2, lat2)
varname = "salinity"
figdir = "./NorthSea/figures/Comparison/2-coeffderiv/"

if not os.path.exists(figdir):
    os.makedirs(figdir)
    
# Select the common depth
depth_common = np.intersect1d(depth1, depth2)

# Loop on depths
for d in depth_common:
    ind1 = np.where(d == depth1)[0]
    ind2 = np.where(d == depth2)[0]
    logger.info((ind1, ind2))
    
    
    field1 = S1[0, ind1, :, :].squeeze()
    field2 = S2[0, ind2, :, :].squeeze()

    mm = "00"
    dd = str(int(d)).zfill(3)

    figname = os.path.join(figdir, "{0}_{1}_{2}".format(varname,mm, dd))
    figtitle = "Annual $-$ {} m".format(d)
    logger.info(figtitle)
    logger.info("Working on {}".format(figname))
    make_plot_2Dfield(llon1, llat1, llon2, llat2, field1, field2, figtitle, 
                      figname, 
                      subtitle1="Without coeff_derivative2",
                      subtitle2="coeff_derivative2 = [0.0, 0.0, 1e-8]")

INFO:ClimComparison:(array([0]), array([0]))
INFO:ClimComparison:Annual $-$ 0.0 m
INFO:ClimComparison:Working on ./NorthSea/figures/Comparison/2-coeffderiv/salinity_00_000
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
INFO:ClimComparison:(array([1]), array([1]))
INFO:ClimComp

## 2. Difference when using different dataset
1. SDC only
2. SDC + World Ocean Atlas

### Read the fields

In [49]:
lon0, lat0, depth0, dates0, T0 = read_climato(temperature_year_sdc0, "temperature")
lon0, lat0, depth0, dates0, S0 = read_climato(salinity_year_sdc0, "salinity")

### Make the comparative plots

In [56]:
llon0, llat0 = np.meshgrid(lon1, lat1)
varname = "salinity"
figdir = "./NorthSea/figures/Comparison/3-sdn_wod/"

if not os.path.exists(figdir):
    os.makedirs(figdir)

# Loop on depths
for idepth, d in enumerate(depth0):
   
    field1 = S0[0, idepth, :, :].squeeze()
    field2 = S1[0, idepth, :, :].squeeze()

    mm = "00"
    dd = str(int(d)).zfill(3)

    figname = os.path.join(figdir, "{0}_{1}_{2}".format(varname,mm, dd))
    figtitle = "Annual $-$ {} m".format(d)
    logger.info(figtitle)
    logger.info("Working on {}".format(figname))
    make_plot_2Dfield(llon0, llat0, llon0, llat0, field1, field2, figtitle, figname,
                     subtitle1="SeaDataNet observations only",
                     subtitle2="SeaDataNet and World Ocean Database")

INFO:ClimComparison:Annual $-$ 0.0 m
INFO:ClimComparison:Working on ./NorthSea/figures/Comparison/3-sdn_wod/salinity_00_000
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
INFO:ClimComparison:Annual $-$ 5.0 m
INFO:ClimComparison:Working on ./NorthSea/figures/Comparison/3-sdn_wo

## Read fields from the old and new products for T and S

In [6]:
lon1, lat1, depth1, dates1, T1 = read_climato(temperature_month_sdn, "temperature")
lon2, lat2, depth2, dates2, T2 = read_climato(temperature_month_sdc, "temperature")
lon1, lat1, depth1, dates1, S1 = read_climato(salinity_month_sdn, "salinity")
lon2, lat2, depth2, dates2, S2 = read_climato(salinity_month_sdc, "salinity")
logger.info(S1.shape)
logger.info(S2.shape)

NameError: name 'temperature_month_sdn' is not defined

In [75]:
llon1, llat1 = np.meshgrid(lon1, lat1)
llon2, llat2 = np.meshgrid(lon2, lat2)

In [78]:
# Select the common depth
depth_common = np.intersect1d(depth1, depth2)

# Loop on depths
for d in depth_common:
    ind1 = np.where(d == depth1)[0]
    ind2 = np.where(d == depth2)[0]
    logger.info((ind1, ind2))
    
    # Loop on month
    for months in range(0, 12):
        field1 = S1[months, ind1, :, :].squeeze()
        field2 = S2[months, ind2, :, :].squeeze()
        
        mm = str(months+1).zfill(2)
        dd = str(int(d)).zfill(3)
        
        
        figname = os.path.join(figdir, "salinity_{0}_{1}".format(mm, dd))
        figtitle = "{} $-$ {} m".format(calendar.month_name[months+1], d)
        logger.info(figtitle)
        logger.info("Working on {}".format(figname))
        make_plot_2Dfield(llon1, llat1, llon2, llat2, field1, field2, figtitle, figname)

INFO:ClimComparison:(array([13]), array([0]))
INFO:ClimComparison:January $-$ 0.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_01_000
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
INFO:ClimCompa

INFO:ClimComparison:August $-$ 30.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_08_030
INFO:ClimComparison:September $-$ 30.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_09_030
INFO:ClimComparison:October $-$ 30.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_10_030
INFO:ClimComparison:November $-$ 30.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_11_030
INFO:ClimComparison:December $-$ 30.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_12_030
INFO:ClimComparison:(array([9]), array([10]))
INFO:ClimComparison:January $-$ 50.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/Se

INFO:ClimComparison:July $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_07_125
INFO:ClimComparison:August $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_08_125
INFO:ClimComparison:September $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_09_125
INFO:ClimComparison:October $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_10_125
INFO:ClimComparison:November $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_11_125
INFO:ClimComparison:December $-$ 125.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/fig

INFO:ClimComparison:June $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_06_300
INFO:ClimComparison:July $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_07_300
INFO:ClimComparison:August $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_08_300
INFO:ClimComparison:September $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_09_300
INFO:ClimComparison:October $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures/Comparison/salinity_10_300
INFO:ClimComparison:November $-$ 300.0 m
INFO:ClimComparison:Working on /home/ctroupin/Projects/SeaDataCloud/Julia/Climatologies/NorthSea/figures